## Assignment 1: R
- GARCIA RODRIGUEZ, EMILIO ALONSO
- PADILLA AQUISE, ALESSANDRO PIERO
- RIEGA NUÑEZ, GABRIEL ANTONIO FERMIN
- SALAMANCA FERNANDEZ, LUCAS PABLO
- SILVA ANDUJAR, NICOLAS


## 1. Data analysis

In [ ]:
install.packages("readr", dependencies = T)
install.packages("dplyr", dependencies = T)
install.packages("tidyr", dependencies = T)
install.packages("psych", dependencies = T)
library(readr)
library(dplyr)
library(tidyr)
library(psych)

1. Import the dataset

In [ ]:
path <- "C:/Users/Emilio/Desktop/Topicos Computacionales/wage2015_subsample_inference.csv"
dt <- read_csv(path)
head(dt)
colnames(data)

2. Display the number of missings (if any) of each variable in the data set


In [ ]:
missings <- colSums(is.na(dt))
print(missings)

3.Report descriptive statistics of the variables (mean, standard deviation, percentiles, etc.). Interpret your results.

In [ ]:
numeric_vars <- dplyr::select(dt, wage, lwage, exp1, exp2, exp3, exp4)
categorical_vars <- dplyr::select(dt, sex, shs, hsg, scl, clg, ad, mw, so, we, ne, occ2, ind2)

In [ ]:
head(numeric_vars)
head(categorical_vars)

In [ ]:
descriptive_stats_numeric <- numeric_vars %>%
  summarise(
    across(
      everything(),
      list(
        mean = ~mean(., na.rm = TRUE),
        standardesv = ~sd(., na.rm = TRUE),
        median = ~median(., na.rm = TRUE),
        q25 = ~quantile(., 0.25, na.rm = TRUE),
        q75 = ~quantile(., 0.75, na.rm = TRUE),
        min = ~min(., na.rm = TRUE),
        max = ~max(., na.rm = TRUE)
      ), .names = "{.col}_{.fn}"
    )) %>%
  pivot_longer(everything(), names_to = c("variable", "estadistica"), names_sep = "_"
  )

psych::describe(numeric_vars) 

In [ ]:
categorical_freq_list <- list()

In [ ]:
for (var in names(categorical_vars)) {
  freq_table <- categorical_vars %>%
    count(!!sym(var)) %>%
    rename(category = !!sym(var), frecuencia = n) %>%
    mutate(variable = var) 
  categorical_freq_list[[var]] <- freq_table
}

In [ ]:
descriptive_stats_categorical <- bind_rows(categorical_freq_list)

In [ ]:
cat("Descriptive Statistics for Numerical Variables:\n")
print(descriptive_stats_numeric)

In [ ]:
cat("\nFrequencies for Categorical Variables:\n")
print(descriptive_stats_categorical)

### Interpretations

#### wage:
The average wage in the sample is 23.41, with a standard deviation of 21.03, reflecting significant variability in earnings. Wages range from a minimum of 3.02 to a maximum of 528.85, indicating a broad spectrum of income levels. The bottom 25th percentile earn less than 13.46, while the median wage is 19.23, and the top 25% earn more than 27.78. The fact that the median is 19.23 suggests the presence of a number of extremely high wage values, which likely skews the distribution.

#### lwage:
The logarithm of wage has a mean of 2.97 and a standard deviation of 0.57, suggesting that the distribution of wages in its logarithmic scale is more symmetric than in its original form. The logarithmic wage values range from a minimum of 1.11 to a maximum of 6.27, with the top 25% of the sample having a log wage greater than 3.32. Also, it has a median of 2.96, that suggest a lower presence of extremely high or low wages in the sample. 

#### sex:
Although sex is a dummy variable, we can see that 44% of the sample are females, while the mode is "male." The standard deviation of 0.5 suggests an almost balanced distribution between males and females.

#### shs:
The "some high school" variable shows that only 2.3% of the sample have some incomplete high school education, which represents a small fraction of the total population. 

#### hsg:
About 24% of the sample has graduated from high school, according to the mean of this variable. The standard deviation of 0.43 reflects moderate diversity in the education levels within the sample, with a significant portion not having attained a college degree. 

#### scl:
Approximately 28% of the respondents have attended some college without necessarily graduating, as indicated by the mean. This suggests that a considerable segment of the population has received some tertiary education, although they may not have completed their studies.

#### clg:
The data shows that 32% of the sample holds a college degree, as reflected in the mean of this variable. This implies that nearly one-third of the sample has completed a college education, which is significant for evaluating human capital.

#### ad:
About 14% of the individuals in the sample have an advanced degree (master's or doctorate), according to the mean of this variable. This relatively low percentage highlights that advanced education is less common compared to basic college education.

#### mw:
The Midwest region represents 26% of the sample, as indicated by the mean of this variable. This suggests that this region has significant representation within the dataset.

#### so:
The South is the most represented region, with 30% of the sample residing in this area, as shown by the mean of this variable. The higher presence of the South may reflect certain geographical trends in the sample.

#### we:
The West region accounts for 22% of the sample, according to the mean. This proportion suggests a relatively equitable distribution among the country's regions, although with less representation compared to the South.

#### ne:
The Northeast region represents 23% of the sample, as indicated by the mean of this variable. This proportion, similar to the other regions, shows a relatively balanced geographical distribution in the dataset.

#### exp1:
The average work experience is 13.76 years, with a standard deviation of 10.61, indicating high variability in the number of years worked among the individuals in the sample. The median is 10 years, meaning that half of the sample has 10 years or less of work experience, while the maximum is 47 years. Also, the mode of work experience is 5.

#### exp2:
The first polynomial of work experience has a mean of 3.02 years, with a standard deviation of 4.00. This suggests that, for certain specific aspects of work experience, most of the sample has relatively low experience.

#### exp3:
The second polynomial of work experience, exp3, has a mean of 8.24 years and a standard deviation of 14.49, indicating a wide variability.

#### exp4:
The third polynomial of work experience, exp4, shows a significantly higher mean of 25.12 years, with a standard deviation of 53.53, again reflecting a high variability.

4.How many women with a college graduate degree (clg) or above have a wage corresponding to the 25% richest of the sample?

In [ ]:
threshold <- quantile(dt$wage, 0.75, na.rm = TRUE) 
print(threshold)
rich_women_25 <- dt %>%
  filter(sex == 1, (clg == 1 | ad == 1), wage > threshold)
rich_women_25_count <- nrow(rich_women_25)
print(rich_women_25_count)
print(rich_women_25)

5. How many men with a high school graduate degree (hsg) or below have a wage corresponding to the 25% richest of the sample? 

In [ ]:
rich_men_25 <- dt %>%
  filter(sex == 0, (hsg == 1 | shs == 1), wage > threshold)
rich_men_25_count <- nrow(rich_men_25)
print(rich_men_25_count)
print(rich_men_25)

6. Create two dataframes. One containing only the log(wage) and the other containig every variable of the data set but the wage related variables.

In [ ]:
#Dataframe only with lwage
df_log_wage <- data.frame(log_wage = log(dt$wage))
print(df_log_wage)
#Dataframe with rest of the variables but the wage related variable
df_no_wage <- dt %>% select(-wage, -lwage)
print(df_no_wage)

## Data wrangling

7. Make an array for our Y variable, which will be the logarithm of wage (lwage column)

In [ ]:
array_Y <- as.array(dt$lwage)
str(array_Y)
print(array_Y) 

8.1 The basic model will have the columns sex hsg scl clg ad so we ne exp1 occ2 ind2

In [ ]:
var_dummys <- model.matrix(~ sex + exp1 + hsg + scl + clg + ad + so + we + ne + occ2 + ind2 - 1, data = dt)
var_dummys_df <- as.data.frame(var_dummys)
var_dummys_array <- as.matrix(var_dummys_df)

In [ ]:
print("Array Basic Model")
print(var_dummys_array)

8.2 The flexible model will have the same columns, and will also include polynomials for experience (exp2 exp3 exp4), as well as the interactions between all experience variables and other variables except for sex

In [ ]:
dt$exp2 <- dt$exp1^2
dt$exp3 <- dt$exp1^3
dt$exp4 <- dt$exp1^4

In [ ]:
dummy_vars <- model.matrix(~ exp1 + exp2 + exp3 + exp4 + hsg + scl + clg + ad + so + we + ne + occ2 + ind2  -1, data = dt)
print(colnames(dummy_vars))

In [ ]:
interaction_list <- list()

In [ ]:
for (exp_var in c("exp1", "exp2", "exp3", "exp4")) {
  for (var in c("hsg", "scl", "clg", "ad", "so", "we", "ne", "occ2", "ind2")) {
    if (var %in% colnames(dummy_vars)) {
      interaction_name <- paste(exp_var, var, sep = "_")
      interaction_list[[interaction_name]] <- dt[[exp_var]] * dummy_vars[, var, drop = FALSE]
    } else {
      warning(paste("Variable", var, "Not found in dummy_vars"))
    }
  }
}

In [ ]:
interaction_df <- do.call(cbind, interaction_list)
colnames(interaction_df) <- paste(names(interaction_list), "interaction", sep = "_")

In [ ]:
flexible_model_df <- cbind(dummy_vars, interaction_df)
flexible_model_array <- as.matrix(flexible_model_df)

In [ ]:
print("Array of the flexible model:")
print(flexible_model_array)

8.3 Create the Array for the Extra-Flexible Model

In [ ]:
var_extraflex <- dt %>%
  select(exp1, exp2, exp3, exp4, hsg, scl, clg, ad, so, we, ne, starts_with("occ2_"), starts_with("ind2_"))

In [ ]:
var_names <- names(var_extraflex)
print(var_names)

In [ ]:
#Make the interactions
interaction_combinations <- expand.grid(var_names, var_names, stringsAsFactors = FALSE)
interaction_combinations <- interaction_combinations[interaction_combinations$Var1 != interaction_combinations$Var2, ]

In [ ]:
for (i in 1:nrow(interaction_combinations)) {
  var1 <- interaction_combinations$Var1[i]
  var2 <- interaction_combinations$Var2[i]
  if (var1 %in% names(var_extraflex) & var2 %in% names(var_extraflex)) {
    interaction_name <- paste(var1, var2, "interaction", sep = "_")
    var_extraflex[[interaction_name]] <- var_extraflex[[var1]] * var_extraflex[[var2]]
  }
}
var_extraflex <- var_extraflex %>% distinct()

In [ ]:
extraflex_array <- as.matrix(var_extraflex)
print("Array modelo extra-flexible:")
print(extraflex_array)

9. Normalization

In [ ]:
dt$lwage_tilde <- (dt$lwage - mean(dt$lwage, na.rm = TRUE)) / sd(dt$lwage, na.rm = TRUE)

In [ ]:
numeric_vars <- c("wage", "exp1", "exp2", "exp3", "exp4", "lwage_tilde")
for (var in numeric_vars) {
  dt[[paste0(var, "_tilde")]] <- (dt[[var]] - mean(dt[[var]], na.rm = TRUE)) / sd(dt[[var]], na.rm = TRUE)
}

In [ ]:
categorical_vars <- c("sex", "hsg", "scl", "clg", "ad", "so", "we", "ne")
for (var in categorical_vars) {
  dt[[paste0(var, "_tilde")]] <- (as.numeric(dt[[var]]) - mean(as.numeric(dt[[var]]), na.rm = TRUE)) / sd(as.numeric(dt[[var]]), na.rm = TRUE)
}

## Linear Regression

10. Split each of the dataframes created (basic, flexible and extra-flexible models) into a training sample (80% of the data) and a test sample.

In [ ]:
set.seed(123)
train_indices <- sample(1:nrow(dt), size = 0.8 * nrow(dt))

In [ ]:
#Basic Model
basic_model_data <- dt[, c("lwage_tilde", "sex_tilde", "exp1_tilde", "hsg_tilde", "scl_tilde", "clg_tilde", "ad_tilde", "so", "we", "ne", "occ2", "ind2")]

In [ ]:
#Split the data withing the train and the test
basic_train <- basic_model_data[train_indices, ]
basic_test <- basic_model_data[-train_indices, ]

In [ ]:
cat("Training sample size (basic model):", nrow(basic_train), "\n")
cat("Test sample size (basic model):", nrow(basic_test), "\n")

In [ ]:
#Flexible model
flexible_model_data <- dt[, c("lwage_tilde", "sex_tilde", "exp1_tilde", "exp2_tilde", "exp3_tilde", "exp4_tilde", "hsg_tilde", "scl_tilde", "clg_tilde", "ad_tilde", "so", "we", "ne", "occ2", "ind2")]

In [ ]:
#Split the data withing the train and the test
flexible_train <- flexible_model_data[train_indices, ]
flexible_test <- flexible_model_data[-train_indices, ]

In [ ]:
cat("Training sample size (flexible model):", nrow(flexible_train), "\n")
cat("Test sample size (flexible model):", nrow(flexible_test), "\n")

In [ ]:
#Extra-flexible Model
extra_flexible_model_data <- dt[, c("lwage_tilde", "sex_tilde", "exp1_tilde", "exp2_tilde", "exp3_tilde", "exp4_tilde", "hsg_tilde", "scl_tilde", "clg_tilde", "ad_tilde", "so", "we", "ne", "occ2", "ind2")]

In [ ]:
#Split the data withing the train and the test
extra_flexible_train <- extra_flexible_model_data[train_indices, ]
extra_flexible_test <- extra_flexible_model_data[-train_indices, ]

In [ ]:
cat("Training sample size (extra-flexible model):", nrow(extra_flexible_train), "\n")
cat("Test sample size (extra-flexible model):", nrow(extra_flexible_test), "\n")

11. Estimate all three models

In [ ]:
# Basi Model
basic_model <- lm(lwage_tilde ~ sex_tilde + exp1_tilde + hsg_tilde + scl_tilde + 
                    clg_tilde + ad_tilde + so + we + ne + occ2 + ind2, 
                  data = basic_train)

In [ ]:
# Flexible Model
flexible_model <- lm(lwage_tilde ~ sex_tilde + exp1_tilde + exp2_tilde + exp3_tilde + 
                       exp4_tilde + hsg_tilde + scl_tilde + clg_tilde + ad_tilde + 
                       so + we + ne + occ2 + ind2, 
                     data = flexible_train)

In [ ]:
# Extra-flexible Model
extra_flexible_model <- lm(lwage_tilde ~ sex_tilde + exp1_tilde + exp2_tilde + exp3_tilde + 
                             exp4_tilde + hsg_tilde + scl_tilde + clg_tilde + ad_tilde + 
                             so + we + ne + occ2 + ind2, 
                           data = extra_flexible_train)

In [ ]:
# Summary 
summary(basic_model)
summary(flexible_model)
summary(extra_flexible_model)

12. Report the MSE and the R 2 for both samples of each model, and the adjusted R 2 for the training sample

In [ ]:
calculate_metrics <- function(model, data, sample_name) {
  predictions <- predict(model, newdata = data)
  actuals <- data$lwage_tilde
  
  mse <- mean((predictions - actuals)^2)
  r_squared <- 1 - (sum((predictions - actuals)^2) / sum((actuals - mean(actuals))^2))
  adjusted_r_squared <- 1 - (1 - r_squared) * (nrow(data) - 1) / (nrow(data) - length(coef(model) - 1))
  
  return(c(MSE = mse, R_squared = r_squared, Adjusted_R_squared = adjusted_r_squared, Sample = sample_name))
}

In [ ]:
# Basic Model
basic_metrics_train <- calculate_metrics(basic_model, basic_train, "Basic - Train")
basic_metrics_test <- calculate_metrics(basic_model, basic_test, "Basic - Test")

In [ ]:
# Flexible Model
flexible_metrics_train <- calculate_metrics(flexible_model, flexible_train, "Flexible - Train")
flexible_metrics_test <- calculate_metrics(flexible_model, flexible_test, "Flexible - Test")

In [ ]:
# Extra-flexible Model
extra_flexible_metrics_train <- calculate_metrics(extra_flexible_model, extra_flexible_train, "Extra-flexible - Train")
extra_flexible_metrics_test <- calculate_metrics(extra_flexible_model, extra_flexible_test, "Extra-flexible - Test")

In [ ]:
#Combine the results
results <- rbind(basic_metrics_train, basic_metrics_test,
                 flexible_metrics_train, flexible_metrics_test,
                 extra_flexible_metrics_train, extra_flexible_metrics_test)

In [ ]:
#Convert to a dataframe for better visualization
results_df <- as.data.frame(results)
print(results_df)

These results indicate that the extra-flexible model suffers from overfitting. This is evident when comparing its R-squared and adjusted R-squared values, which drop significantly from 0.35 to -0.23. This drastic reduction suggests that adding more regressors does not enhance the model's fit, a common sign of overfitting. In contrast, the basic model demonstrates the best out-of-sample performance, as it has the lowest out-of-sample MSE (0.69) and the highest out-of-sample R-squared (0.31) among the models.